In [1]:
import pandas as pd
model_type ='roberta'
from utils import *

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import itertools 
import numpy as np

In [4]:
model_type = 'roberta'
add_response_token = False

In [5]:
csv_data = pd.read_csv('add_features_train.tsv', sep='\t')
csv_data.drop_duplicates(['context', 'phrase'], inplace=True)
csv_data['context'] = csv_data['context'].apply(lambda x: eval(x))
feature_cols = csv_data.columns[3:]

if model_type == 'bert':
    csv_data['prep_context'] = csv_data['context'].apply(lambda x: preprocess_context(x, 'bert'))
    if add_response_token:
        add_token = '[RESPONSE_TOKEN]'
        csv_data['prep_phrase_context'] = csv_data['prep_context'] + add_token + '— ' + csv_data['phrase'] + ' [SEP]'
    else:
        csv_data['prep_phrase_context'] = csv_data['prep_context'] + '— ' + csv_data['phrase'] + ' [SEP]'
elif model_type == 'roberta':
    csv_data['prep_context'] = csv_data['context'].apply(lambda x: preprocess_context(x, 'roberta'))
    if add_response_token:
        add_token = '[RESPONSE_TOKEN]'
        csv_data['prep_phrase_context'] = csv_data['prep_context'] + add_token + '– ' + csv_data['phrase'] + '</s>'
    else:
        csv_data['prep_phrase_context'] = csv_data['prep_context'] + '– ' + csv_data['phrase'] + '</s>'
elif model_type == 't5':
    csv_data['prep_context'] = csv_data['context'].apply(lambda x: preprocess_context(x, 't5'))
    csv_data['prep_phrase_context'] = csv_data['prep_context'] + ' <extra_id_0> ' + '— ' + csv_data['phrase'] + ' <extra_id_0>'

# if not add_features:
#     csv_data = csv_data[['prep_phrase_context', 'label']]
# else:
#     csv_data = csv_data[['prep_phrase_context', 'label'] + list(feature_cols)]
print(csv_data)

train_data, val_data = train_test_split(csv_data, test_size=0.3, random_state=42, shuffle=True, stratify=csv_data['label'])
test_data, val_data = train_test_split(val_data, test_size=0.66, random_state=42, shuffle=True, stratify=val_data['label'])
bad_train_data = train_data[(train_data['label'] == 1) & (train_data['relevance'] < 0.3) & (~train_data['phrase'].str.contains('марус|помощн'))] 
bad_train_data = bad_train_data[(~bad_train_data['prep_context'].isin(val_data['prep_context'].unique())) & (~bad_train_data['prep_context'].isin(test_data['prep_context'].unique()))]

bad_contexts = bad_train_data['context']
bad_phrases = bad_train_data['phrase']

random_pairs = list(itertools.product(bad_contexts, bad_phrases))
random_10k_idx = np.random.randint(len(random_pairs), size=10_000)
random_pairs_10k = np.array(random_pairs)[random_10k_idx]

                                                  phrase  \
0                                        Ну, куда пошли?   
1              А сколько будет миллиард долларов рублей?   
2      Марусь включи, пожалуйста, Елена Вальяк. Гипно...   
3             Да, о том, что я тебя очень уважаю, люблю.   
4                                       Детскую песенку.   
...                                                  ...   
28616                                   Так где мой нож?   
28617                                            Дальше.   
28618                        Маруся, я очень тебя люблю.   
28619                   Голову вас зафиксировала камера.   
28620                                  Что представляем?   

                                                 context  label  \
0      [{'user': '', 'marusia': ''}, {'user': '', 'ma...      1   
1      [{'user': 'А сколько будет?', 'marusia': 'Изви...      0   
2      [{'user': 'Маруся, через десять минут будильни...      0   
3      [{'u

/tmp/ipykernel_657240/2520559932.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  random_pairs_10k = np.array(random_pairs)[random_10k_idx]


In [6]:
bad_contexts = bad_contexts.reset_index()
bad_phrases = bad_phrases.reset_index()

In [7]:
user_replics_cols = ['TTR', 'Количество абзацев', 'Количество слогов', 'Количество слов']
phrase_cols = ['Среднее количество слов в предложении', 'Существительных', 'Количество слогов', 'RR', ]

In [8]:
user_replics_cols = [column + ' контекста юзера' for column in user_replics_cols] + ['toxicity_user_context']
phrase_cols = [column + ' таргетной фразы' for column in phrase_cols] + ['toxicity_phrase', 'linguistic_acceptability']

In [9]:
train_data.reset_index(inplace=True)

In [10]:
bad_contexts = bad_contexts.merge(train_data[['index'] + user_replics_cols], on='index')

In [11]:
bad_phrases = bad_phrases.merge(train_data[['index'] + phrase_cols], on='index')

In [12]:
bad_train_data_merg = pd.merge(bad_contexts, bad_phrases, on='index')

In [13]:
bad_train_data_merg['str_for_resp_qual'] = bad_train_data_merg.apply(return_str_for_resp_qual, axis=1)
bad_train_data_merg['resp_qual'] = bad_train_data_merg['str_for_resp_qual'].apply(lambda x: predict_resp_qual(x))


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


ValueError: could not convert string to float: '.'

In [15]:
bad_train_data_merg['relevance'] = bad_train_data_merg['resp_qual'].apply(lambda x: x.split()[0]).astype('float64')
bad_train_data_merg['specificity'] = bad_train_data_merg['resp_qual'].apply(lambda x: x.split()[1]).astype('float64')
bad_train_data_merg.drop(['resp_qual', 'str_for_resp_qual'], axis=1, inplace=True)

In [17]:
bad_train_data_merg.shape == train_data.shape

False

In [26]:
bad_train_data_merg['label'] = 1

In [20]:
bad_train_data_merg.shape

(2058, 16)

In [21]:
train_data.columns.shape

(19,)

In [31]:
bad_train_data_merg.to_csv('sampled_data.tsv', sep='\t')

In [1]:
import pandas as pd

In [27]:
set(train_data.columns) - set(bad_train_data_merg.columns)

{'prep_context', 'prep_phrase_context'}

In [29]:
bad_train_data_merg

,index,context,TTR контекста юзера,Количество абзацев контекста юзера,Количество слогов контекста юзера,Количество слов контекста юзера,toxicity_user_context,phrase,Среднее количество слов в предложении таргетной фразы,Существительных таргетной фразы,Количество слогов таргетной фразы,RR таргетной фразы,toxicity_phrase,linguistic_acceptability,relevance,specificity,label
0,9204,[{'user': 'Алиса Маруся. А что такое Алиса и М...,0.67,3.0,31.0,15.0,0.048968,"Маруся, доброе утро.",3.0,2.0,8.0,0.33,0.032900,0.468607,0.220123,0.277094,1
1,503,"[{'user': 'Стоп.', 'marusia': ''}, {'user': 'С...",0.88,5.0,36.0,16.0,0.095805,Таблица.,1.0,1.0,3.0,1.00,0.008617,0.555115,0.267837,0.089914,1
2,2941,"[{'user': 'Ой, а с тобой можно развлечься?', '...",0.86,5.0,31.0,21.0,0.031558,Ой ой ой ой ой.,5.0,1.0,5.0,0.68,0.012027,0.354453,0.208209,0.062512,1
3,21278,"[{'user': 'Ты лох, объелся блох.', 'marusia': ...",0.58,4.0,22.0,12.0,0.506419,Ты дура.,2.0,0.0,3.0,0.50,0.998014,0.320057,0.152814,0.057829,1
4,12632,"[{'user': '', 'marusia': ''}, {'user': 'Включи...",0.85,5.0,26.0,13.0,0.014131,"Не молодайте, используют и ноги.",5.0,1.0,12.0,0.20,0.043489,0.511057,0.016425,0.018904,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,2754,"[{'user': 'Маруся, найди мне всякие игры про т...",0.91,5.0,50.0,23.0,0.184574,Стоп.,1.0,1.0,1.0,1.00,0.032472,0.376658,0.257003,0.453475,1
2054,21449,"[{'user': '', 'marusia': ''}, {'user': 'Маруся...",0.94,4.0,40.0,18.0,0.054325,С Марусей болтаю.,3.0,2.0,6.0,0.33,0.029635,0.402029,0.019811,0.034381,1
2055,21989,"[{'user': 'Сколько?', 'marusia': 'Сколько зубо...",1.00,4.0,29.0,13.0,0.045936,А.,1.0,1.0,1.0,1.00,0.017397,0.517861,0.071843,0.014597,1
2056,135,"[{'user': 'Гав.', 'marusia': 'Гав гав'}, {'use...",0.85,5.0,23.0,13.0,0.097030,Маруся.,1.0,1.0,3.0,1.00,0.042738,0.573069,0.024122,0.016275,1


In [28]:
pd.merge(bad_train_data_merg, train_data[['context']])

,index,context,TTR контекста юзера,Количество абзацев контекста юзера,Количество слогов контекста юзера,Количество слов контекста юзера,toxicity_user_context,phrase,Среднее количество слов в предложении таргетной фразы,Существительных таргетной фразы,Количество слогов таргетной фразы,RR таргетной фразы,toxicity_phrase,linguistic_acceptability,relevance,specificity,label
0,9204,[{'user': 'Алиса Маруся. А что такое Алиса и М...,0.67,3.0,31.0,15.0,0.048968,"Маруся, доброе утро.",3.0,2.0,8.0,0.33,0.032900,0.468607,0.220123,0.277094,1
1,503,"[{'user': 'Стоп.', 'marusia': ''}, {'user': 'С...",0.88,5.0,36.0,16.0,0.095805,Таблица.,1.0,1.0,3.0,1.00,0.008617,0.555115,0.267837,0.089914,1
2,2941,"[{'user': 'Ой, а с тобой можно развлечься?', '...",0.86,5.0,31.0,21.0,0.031558,Ой ой ой ой ой.,5.0,1.0,5.0,0.68,0.012027,0.354453,0.208209,0.062512,1
3,21278,"[{'user': 'Ты лох, объелся блох.', 'marusia': ...",0.58,4.0,22.0,12.0,0.506419,Ты дура.,2.0,0.0,3.0,0.50,0.998014,0.320057,0.152814,0.057829,1
4,12632,"[{'user': '', 'marusia': ''}, {'user': 'Включи...",0.85,5.0,26.0,13.0,0.014131,"Не молодайте, используют и ноги.",5.0,1.0,12.0,0.20,0.043489,0.511057,0.016425,0.018904,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,2754,"[{'user': 'Маруся, найди мне всякие игры про т...",0.91,5.0,50.0,23.0,0.184574,Стоп.,1.0,1.0,1.0,1.00,0.032472,0.376658,0.257003,0.453475,1
2054,21449,"[{'user': '', 'marusia': ''}, {'user': 'Маруся...",0.94,4.0,40.0,18.0,0.054325,С Марусей болтаю.,3.0,2.0,6.0,0.33,0.029635,0.402029,0.019811,0.034381,1
2055,21989,"[{'user': 'Сколько?', 'marusia': 'Сколько зубо...",1.00,4.0,29.0,13.0,0.045936,А.,1.0,1.0,1.0,1.00,0.017397,0.517861,0.071843,0.014597,1
2056,135,"[{'user': 'Гав.', 'marusia': 'Гав гав'}, {'use...",0.85,5.0,23.0,13.0,0.097030,Маруся.,1.0,1.0,3.0,1.00,0.042738,0.573069,0.024122,0.016275,1
